In [ ]:

import os
from google.colab import drive

drive.mount('/content/drive')

zip_path = '/content/drive/MyDrive/DiaChestXR_Project/processed.zip'

if not os.path.exists('/content/data'):
    print("Unzipping data...")
    !unzip -q "$zip_path" -d /content/
    print("Unzip completed.")
else:
    print("Data already unzipped.")


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import pandas as pd
from PIL import Image

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class ChestXrayDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None, mode='dense'):

        self.img_dir = img_dir
        self.transform = transform
        self.mode = mode

        df = pd.read_csv(csv_file)
        self.classes = sorted(df['class_name'].unique())
        self.c2i = {c: i for i, c in enumerate(self.classes)}

        if mode == 'dense':
            self.data = df.groupby('image_id')['class_name'].apply(list).reset_index()
        else:
            self.data = df

    def __len__(self): return len(self.data)

    def __getitem__(self, idx):
        if self.mode == 'dense':
            row = self.data.iloc[idx]
            img_id = row['image_id']
            labels = row['class_name']
        else:
            row = self.data.iloc[idx]
            img_id = row['image_id']
            label_name = row['class_name']

        exts = ['.jpg', '.png', '.jpeg']
        for ext in exts:
            path = os.path.join(self.img_dir, str(img_id) + ext)
            if os.path.exists(path):
                img_path = path
                break

        try:
            image = Image.open(img_path).convert("RGB")
        except:
            image = Image.new('RGB', (227, 227))

        if self.transform: image = self.transform(image)

        if self.mode == 'dense':
            target = torch.zeros(len(self.classes))
            for cls in labels:
                if cls in self.c2i: target[self.c2i[cls]] = 1.0
        else:
            target = torch.tensor(self.c2i[label_name], dtype=torch.long)

        return image, target

# --- TRAINING ENGINE ---
def train_on_colab(model_type, epochs=10):
    print(f"\nStarting training: {model_type.upper()}")


    data_root = "/content/processed"
    img_dir = f"{data_root}"
    csv_train = f"{data_root}/splits/train_{model_type}.csv"
    csv_val = f"{data_root}/splits/val_{model_type}.csv"

    # Transform
    tf = transforms.Compose([
        transforms.Resize((227, 227)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    # Load Data
    try:
        train_ds = ChestXrayDataset(csv_train, img_dir, tf, mode=model_type)
        val_ds = ChestXrayDataset(csv_val, img_dir, tf, mode=model_type)

        train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)
        print(f"Data Loaded: {len(train_ds)} Train | {len(val_ds)} Val")
        print(f"Classes: {train_ds.classes}")
    except Exception as e:
        print(f"Error loading data: {e}")
        return

    # Init Model
    num_classes = len(train_ds.classes)
    if model_type == 'dense':
        print("Model: DenseNet121")
        model = models.densenet121(weights='IMAGENET1K_V1')
        model.classifier = nn.Linear(1024, num_classes)
        criterion = nn.BCEWithLogitsLoss()
    else:
        print("Model: AlexNet")
        model = models.alexnet(weights='IMAGENET1K_V1')
        for param in model.features.parameters(): param.requires_grad = False
        model.classifier[6] = nn.Linear(4096, num_classes)
        criterion = nn.CrossEntropyLoss()

    model.to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)

    # Loop
    best_acc = 0.0
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0

        # Train
        for imgs, targets in train_loader:
            imgs, targets = imgs.to(DEVICE), targets.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Validation
        model.eval()
        correct = 0; total = 0
        with torch.no_grad():
            for imgs, targets in val_loader:
                imgs, targets = imgs.to(DEVICE), targets.to(DEVICE)
                outputs = model(imgs)

                if model_type == 'dense':
                    preds = (torch.sigmoid(outputs) > 0.5).float()
                    correct += (preds == targets).sum().item()
                    total += targets.numel()
                else:
                    _, preds = torch.max(outputs, 1)
                    correct += (preds == targets).sum().item()
                    total += targets.size(0)

        val_acc = correct / total * 100
        print(f"Epoch {epoch+1}: Loss={train_loss/len(train_loader):.4f} | Val Acc={val_acc:.2f}%")

        if val_acc > best_acc:
            best_acc = val_acc
            save_path = f"/content/drive/MyDrive/DiaChestXR_Project/{model_type}_best.pth"
            torch.save(model.state_dict(), save_path)
            print(f"Saved Best Model: {save_path}")



train_on_colab('alex', epochs=15)

train_on_colab('dense', epochs=15)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
⏳ Đang giải nén... (Chờ xíu nhé)
replace /content/processed/000434271f63a053c4128a0ba6352c7f.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
✅ Đã giải nén xong tại /content/data

🚀 BẮT ĐẦU TRAIN: ALEX
✅ Data Loaded: 6861 Train | 1716 Val
🏷️ Classes: ['COVID-19', 'Normal', 'Pneumonia']
🏗️ Model: AlexNet
Epoch 1: Loss=0.0597 | Val Acc=99.77%
   💾 Saved Best Model: /content/drive/MyDrive/DiaChestXR_Project/alex_best.pth
Epoch 2: Loss=0.0229 | Val Acc=99.71%
Epoch 3: Loss=0.0163 | Val Acc=99.71%
Epoch 4: Loss=0.0169 | Val Acc=99.71%
Epoch 5: Loss=0.0094 | Val Acc=99.59%
Epoch 6: Loss=0.0087 | Val Acc=99.88%
   💾 Saved Best Model: /content/drive/MyDrive/DiaChestXR_Project/alex_best.pth
Epoch 7: Loss=0.0096 | Val Acc=99.30%
Epoch 8: Loss=0.0105 | Val Acc=99.88%
Epoch 9: Loss=0.0078 | Val Acc=99.88%
Epoch 10: Loss=0.0121 | Val Acc=99.65%
Epoch 11: Loss=0.0077 | Val Acc

100%|██████████| 30.8M/30.8M [00:00<00:00, 112MB/s]


Epoch 1: Loss=0.1954 | Val Acc=94.57%
   💾 Saved Best Model: /content/drive/MyDrive/DiaChestXR_Project/dense_best.pth
Epoch 2: Loss=0.1340 | Val Acc=94.82%
   💾 Saved Best Model: /content/drive/MyDrive/DiaChestXR_Project/dense_best.pth
Epoch 3: Loss=0.1200 | Val Acc=95.14%
   💾 Saved Best Model: /content/drive/MyDrive/DiaChestXR_Project/dense_best.pth
Epoch 4: Loss=0.1104 | Val Acc=94.90%
Epoch 5: Loss=0.1004 | Val Acc=95.26%
   💾 Saved Best Model: /content/drive/MyDrive/DiaChestXR_Project/dense_best.pth
Epoch 6: Loss=0.0919 | Val Acc=95.20%
Epoch 7: Loss=0.0844 | Val Acc=95.17%
Epoch 8: Loss=0.0786 | Val Acc=95.21%
Epoch 9: Loss=0.0689 | Val Acc=95.21%
Epoch 10: Loss=0.0618 | Val Acc=95.01%
Epoch 11: Loss=0.0530 | Val Acc=94.85%
Epoch 12: Loss=0.0467 | Val Acc=95.26%
Epoch 13: Loss=0.0404 | Val Acc=95.17%
Epoch 14: Loss=0.0361 | Val Acc=95.17%
Epoch 15: Loss=0.0306 | Val Acc=95.16%
